In [1]:
from decouple import config
from openai import OpenAI

In [3]:
сlient = OpenAI(api_key=config('OPENAI_API_KEY'))

my_assistant = сlient.beta.assistants.create(
    instructions="empty prompt",
    name='Seller',
    tools=[],
    model="gpt-4o",
)

print(my_assistant)

Assistant(id='asst_pINUE7tmo4Kld4bWMZOtDajO', created_at=1752746339, description=None, instructions='empty prompt', metadata={}, model='gpt-4o', name='Seller', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None)


In [ ]:
import pprint
pprint.pprint(my_assistant.model_dump())

{'created_at': 1752746339,
 'description': None,
 'id': 'asst_pINUE7tmo4Kld4bWMZOtDajO',
 'instructions': 'empty prompt',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Seller',
 'object': 'assistant',
 'reasoning_effort': None,
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'tools': [],
 'top_p': 1.0}


In [8]:
def update_env_variable(key, value, env_file=".env"):
    lines = []
    updated = False

    with open(env_file, "r") as f:
        for line in f:
            if line.strip().startswith(f"{key}="):
                lines.append(f"{key}={value}\n")
                updated = True
            else:
                lines.append(line)

    if not updated:
        lines.append(f"{key}={value}\n")

    with open(env_file, "w") as f:
        f.writelines(lines)

update_env_variable("ASSISTANT_ID", my_assistant.model_dump()['id'])

In [10]:
import sqlite3

# === Инициализация базы данных ===
conn = sqlite3.connect("users.db", check_same_thread=False)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY,
    username TEXT,
    thread_id TEXT
)
""")
conn.commit()

In [11]:
# Подключаемся к существующей базе
conn = sqlite3.connect("users.db")
cur = conn.cursor()

# Список тестовых пользователей (user_id, username, thread_id)
users = [
    (1111111, "user_one", "thread_abc123"),
    (2222222, "user_two", "thread_def456"),
    (3333333, "user_three", "thread_xyz789"),
]

# Вставка данных
cur.executemany(
    "INSERT OR REPLACE INTO users (user_id, username, thread_id) VALUES (?, ?, ?)",
    users
)

# Сохраняем изменения и закрываем соединение
conn.commit()
conn.close()

print("Три пользователя успешно добавлены в базу данных.")

Три пользователя успешно добавлены в базу данных.


In [12]:
def add_user_to_db(user_id: int, username: str = "") -> str:
    conn = sqlite3.connect("users.db")
    cur = conn.cursor()

    # Проверим, есть ли уже такой пользователь
    cur.execute("SELECT thread_id FROM users WHERE user_id = ?", (user_id,))
    existing = cur.fetchone()
    if existing:
        conn.close()
        return existing[0]  # Уже есть, возвращаем существующий thread_id

    # Создаём новый thread
    # thread = client.beta.threads.create()

    # Добавляем пользователя
    cur.execute(
        "INSERT INTO users (user_id, username, thread_id) VALUES (?, ?, ?)",
        (user_id, username, 'thread_id')
    )
    conn.commit()
    conn.close()

In [14]:
new_thread_id = add_user_to_db(user_id=4444444, username="user_four")
print(f"Добавлен новый пользователь с thread_id: {new_thread_id}")

Добавлен новый пользователь с thread_id: None


In [1]:
from pyrogram import Client
from decouple import config

bot = Client(
    name=config('LOGIN'),
    api_id=config('API_ID'),
    api_hash=config('API_HASH'),
    phone_number=config('PHONE')
)
bot.start()

UndefinedValueError: LOGIN not found. Declare it as envvar or define a default value.

In [ ]:
from database import*
from pyrogram import Client
from decouple import Config, RepositoryEnv
import os

config = Config(RepositoryEnv('/home/appuser/Seller/.env'))

bot = Client(
    name=config('LOGIN'),
    api_id=config('API_ID'),
    api_hash=config('API_HASH'),
    phone_number=config('PHONE')
)
bot.start()

await bot.connect()
phone = '+79936321517'
sCode = await bot.send_code(phone)
code = input('Enter auth code: ')
await bot.sign_in(phone, sCode.phone_code_hash, code)

<coroutine object Start.start at 0x7f703b56a260>

In [1]:
from pyrogram import Client
from decouple import Config, RepositoryEnv

config = Config(RepositoryEnv('/home/appuser/Seller/.env'))

bot = Client(
    name=config('LOGIN'),
    api_id=config('API_ID'),
    api_hash=config('API_HASH'),
    phone_number=config('PHONE')
)

print(config('PHONE'))

+79376371117


In [82]:
from pyrogram.raw import functions, types
from pyrogram import Client
from decouple import Config, RepositoryEnv

config = Config(RepositoryEnv('/home/appuser/Seller/.env'))

bot = Client(
    name=config('LOGIN'),
    api_id=config('API_ID'),
    api_hash=config('API_HASH'),
    phone_number=config('PHONE')
)

In [5]:
async with bot:
    # Используем raw API для создания InputUser
    input_user = types.InputUser(user_id=7312739864 , access_hash=203196834816003768)
    
    result = await bot.invoke(
        functions.users.GetUsers(id=[input_user])
    )
    
    user = result[0]
    print(user.id, user.first_name, user.username, user.phone, user.access_hash)

7312739864 Александр major_pt None 203196834816003768


In [89]:
async with bot:
    user = await bot.get_users(778911974)
    await bot.send_dice(778911974)

In [97]:
from database import*

async with bot:
    init_db()

    delete_user_by_name('Endergosha')
    # delete_user_by_name('abracadabra12331')
    # delete_user_by_name('AleksnotAleks')
    # delete_user_by_name('nickoraguaua')

    await add_user_by_name(bot, 'Endergosha')
    # await add_user_by_name(bot, 'abracadabra12331')
    # await add_user_by_name(bot, 'AleksnotAleks')
    # await add_user_by_name(bot, 'nickoraguaua')
    # await add_user_by_name(bot, 'fly_paragliding')

    show_users(sorted_by_contact=True)

+-----------+---------------+------------+-------------+--------+-----------+----------+----------------+-------------+---------------------------+-----------+
|        Id | Access Hash   | Username   | Telephone   | Name   |   Contact |   Banned |   Added to CRM | Thread ID   | Info                      | Summary   |
+===========+===============+============+=============+========+===========+==========+================+=============+===========================+===========+
| 778911974 |               | Endergosha |             |        |         0 |        0 |              0 |             | TG TELEPHONE NUBMER: None |           |
+-----------+---------------+------------+-------------+--------+-----------+----------+----------------+-------------+---------------------------+-----------+


In [98]:
show_users(sorted_by_contact=True)

+-----------+---------------------+------------+-------------+--------+-----------+----------+----------------+---------------------------------+---------------------------+-----------+
|        Id |         Access Hash | Username   | Telephone   | Name   |   Contact |   Banned |   Added to CRM | Thread ID                       | Info                      | Summary   |
+===========+=====================+============+=============+========+===========+==========+================+=================================+===========================+===========+
| 778911974 | 1474220942745780297 | Endergosha |             |        |         0 |        0 |              0 | thread_mLcXHNq7woY3ZWs6dAa9bVF1 | TG TELEPHONE NUBMER: None |           |
+-----------+---------------------+------------+-------------+--------+-----------+----------+----------------+---------------------------------+---------------------------+-----------+
